In [ ]:
%reload_ext autoreload
%autoreload 2

import functools

print = functools.partial(print, flush=True)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, sta, find_depth_neurons
from cottage_analysis.plotting import sta_plots, basic_vis_plots


In [ ]:
project='hey2_3d-vision_foodres_20220101'
session_name='PZAH10.2f_S20230727'
flexilims_session = flz.get_flexilims_session(project)
# vs_df_all, trials_df_all = spheres.sync_all_recordings(
#     session_name=session_name,
#     flexilims_session=flexilims_session,
#     project=project,
#     filter_datasets={"anatomical_only": 3},
#     recording_type="two_photon",
#     protocol_base="SpheresPermTubeReward",
#     photodiode_protocol=5,
#     return_volumes=True,
# )
# Regenerating frames for all recordings
frames_all, imaging_df_all = spheres.regenerate_frames_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
    resolution=5,
)


In [ ]:
from cottage_analysis.analysis import find_depth_neurons, pipeline_utils
neurons_ds = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    conflicts="skip",
)
neurons_df = pd.read_pickle(neurons_ds.path_full)


coef, r2, best_reg_xys, best_reg_depths  = spheres.fit_3d_rfs_hyperparam_tuning(imaging_df_all, 
                                                frames_all[:,:,int(frames_all.shape[2]//2):], 
                                                reg_xys=[20, 40, 80, 160, 320], 
                                                reg_depths=[20, 40, 80, 160, 320], 
                                                shift_stims=2, 
                                                use_col="dffs", 
                                                k_folds=5, 
                                                tune_separately=True, 
                                                validation=False)

coef_ipsi, r2_ipsi = spheres.fit_3d_rfs_ipsi(
    imaging_df_all,
    frames_all[:,:,:int(frames_all.shape[2]//2)],
    best_reg_xys,
    best_reg_depths,
    shift_stims=2,
    use_col="dffs",
    k_folds=5,
    validation=False,
)


In [ ]:
roi = 179
stas = sta.sta(frames_all, np.concatenate(imaging_df_all.spks))
sta_plots.plot_sta(stas, roi)
plt.colorbar()

In [ ]:
verbose=False
imaging_df = imaging_df_all
dffs = np.concatenate(imaging_df_all.dffs)
delays = [-1/15*4, -1/15*2, -1/15*1,0, 1/15*1, 1/15*2, 1/15*4]
frame_rate=15

full_stas, ndelays, ndepths = sta.sta_by_depth(frames=frames_all, dffs=np.concatenate(imaging_df_all.dffs), imaging_df=imaging_df_all, delays=delays, frame_rate=15, verbose=True)

    

In [ ]:
plt.figure(figsize=(3*len(delays),15))
for idelay in range(len(delays)):
    this_sta = full_stas[idelay,roi,:,:,:]
    vmax = np.quantile(this_sta[np.isfinite(this_sta)], .999)
    for i in range(5):  
        plt.subplot2grid((5,len(delays)), (i,idelay))
        plt.imshow(this_sta[i], origin='lower', vmin=-vmax, vmax=vmax, cmap='bwr')
        plt.colorbar()

In [ ]:
stimulus_frames = np.sum(frames, axis=(1,2)) > 0
dffs = np.concatenate(imaging_df.spks)
dffs = dffs - np.mean(dffs[stimulus_frames, :], axis=0)
stas = dffs.T @ np.roll(np.reshape(frames, (frames.shape[0], -1)), 1, axis=0)
stas = np.reshape(stas, (stas.shape[0], *frames.shape[1:]))
sum_frames = np.sum(frames, axis=0)
roi = 179
sta = stas[roi, :, :] / sum_frames
vmax = np.quantile(sta[np.isfinite(sta)], .999)
plt.imshow(sta, origin='lower', vmin=-vmax, vmax=vmax, cmap='bwr')

In [ ]:
sig, sig_ipsi = spheres.find_sig_rfs(coef, coef_ipsi, n_std=5)
np.mean(sig), np.mean(sig_ipsi)


In [ ]:
select_rois = [352, 456, 272, 281, 285, 287, 531, 539, 530]
# [352, 456, 272, 281, 285, 287, 531, 539, 530]
plt.figure(figsize=(3*len(select_rois),3*6))
depth_list = find_depth_neurons.find_depth_list(trials_df_all)
coef_mean = np.mean(np.stack(coef, axis=2), axis=2)
fitted_sta = coef_mean[:-1,:].reshape(5, 16, 24, coef_mean.shape[1])
    
for iroi, roi in enumerate(select_rois):  
    # sta_plots.basic_vis_sta(coef, neurons_df, trials_df, depth_list, frames, roi, plot_rows=6, plot_cols=1, plot_y=0, fontsize_dict={"title": 10, "tick": 10, "label": 10})
    vmax = np.quantile(fitted_sta[:,:,:,roi], .999)

    plt.subplot2grid((6,len(select_rois)), (0,iroi), rowspan=1, colspan=1)
    basic_vis_plots.plot_depth_tuning_curve(
        neurons_df=neurons_df,
        trials_df=trials_df_all,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=3,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        fontsize_dict={"title": 10, "tick": 10, "label": 10},
    )
    
    for i in range(len(depth_list)):
        plt.subplot2grid((6,len(select_rois)), (i+1,iroi), rowspan=1, colspan=1)
        plt.imshow(fitted_sta[i,:,:,roi], origin='lower', vmax=vmax, vmin=-vmax, cmap='bwr', extent=[0,120,-40,40], aspect='equal')
        # plt.colorbar()
    
        if i==0:
            plt.title(f"best xy {best_reg_xys[roi]}, best depth {best_reg_depths[roi]}")
plt.tight_layout()    
# VERSION=1
# save_root = '/camp/lab/znamenskiyp/home/shared/presentations/SfN2023/ver'+str(VERSION)+'/STA/'
# import os
# os.makedirs(save_root, exist_ok=True)
# plt.savefig(save_root+"example_stas.pdf", bbox_inches="tight")


In [ ]:
plt.imshow(np.mean(r2s[:,:,:,1]>.01, axis=2), origin='lower')
plt.xticks(range(len(reg_depth)), reg_depth)
plt.yticks(range(len(reg_xy)), reg_xy)
plt.xlabel('reg_depth')
plt.ylabel('reg_xy')
plt.colorbar()
plt.title('Fraction of neurons with R2>0.01')